# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
import numpy as np
from sklearn import cross_validation, tree, datasets, metrics
from sklearn.model_selection import train_test_split

/home/aleksei/.virtualenvs/plots/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [3]:
base_algorithms_list = []
coefficients_list = []

dataset = datasets.load_boston()
X = dataset.data
y = dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

In [4]:
def gbm_predict(X):
    """
    Предсказывает значения из X с помощью композиции алгоритмов
    """
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [5]:
def write_ans(number: int, ans):
    with open("ans{}".format(number), 'w') as f:
        f.write(str(ans))

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [6]:
base_algorithms_list = []
coefficients_list = []

# Zero prediction class
class A_0:
    def fit(self, X, y):
        self.n_features = X.size
        self.n_targets = y.size
    def predict(self, X):
        return np.zeros(self.n_targets)
    
# Zero iteration
a_0 = A_0()
a_0.fit(X_train, y_train)
base_algorithms_list.append(a_0)
coefficients_list.append(0.9)

In [7]:
def teach_model(decrease_coeffs=False):
    for N in range(1, 51):
        s = -(np.array(gbm_predict(X_train)) - y_train)
        b_N = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
        b_N.fit(X_train, s)
        base_algorithms_list.append(b_N)
        if not decrease_coeffs:
            coefficients_list.append(0.9)   # 0.9 coeff for all
        else:
            coefficients_list.append(0.9/(1.0 + N))

In [8]:
%time teach_model()

CPU times: user 26.5 s, sys: 4 ms, total: 26.6 s
Wall time: 26.5 s


In [9]:
ans2 = metrics.mean_squared_error(gbm_predict(X_test), y_test)**0.5
print("RMSE was {}".format(ans2))
write_ans(2, ans2)

RMSE was 5.476650974168953


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [10]:
%time teach_model(decrease_coeffs=True)

CPU times: user 1min 22s, sys: 52 ms, total: 1min 22s
Wall time: 1min 22s


In [11]:
ans3 = metrics.mean_squared_error(gbm_predict(X_test), y_test)**0.5
print("RMSE was {}".format(ans3))
write_ans(3, ans3)

RMSE was 5.4766495845979914


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [12]:
import xgboost as xgb
from sklearn import grid_search

/home/aleksei/.virtualenvs/plots/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [13]:
regressor = xgb.XGBRegressor(n_jobs=8)
regressor.get_params().keys()
grid = {
    'n_estimators': [10, 50, 100, 1000],
    'learning_rate': np.hstack((np.linspace(0.01, 0.1, 10), [0.9])),
    'max_depth': [1, 2, 5, 10, 100]
}

In [14]:
grid_cv = grid_search.GridSearchCV(regressor, grid, scoring='neg_mean_absolute_error', cv = 3)

In [15]:
%time grid_cv.fit(X_train, y_train)

CPU times: user 7min 25s, sys: 2.04 s, total: 7min 27s
Wall time: 57.6 s


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=8, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100, 1000], 'max_depth': [1, 2, 5, 10, 100], 'learning_rate': array([ 0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,  0.09,
        0.1 ,  0.9 ])},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [16]:
print(grid_cv.best_estimator_)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.070000000000000007,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=8, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)


In [21]:
metrics.mean_squared_error(grid_cv.predict(X_test), y_test)**0.5

5.0775433379880495

In [22]:
for n_est in [10, 50, 100, 1000]:
    for depth in [1, 2, 5, 10, 100]:
        regressor1 = xgb.XGBRegressor(max_depth=depth, n_estimators=n_est, n_jobs=8)
        regressor1.fit(X_train, y_train)
        error = metrics.mean_squared_error(regressor1.predict(X_test), y_test)
        print("Regressor:\n n_estimators is {}\n depth is {}\n MSE is {}"
              .format(n_est, depth, error))

Regressor:
 n_estimators is 10
 depth is 1
 MSE is 26.884424633275902
Regressor:
 n_estimators is 10
 depth is 2
 MSE is 26.979710878069866
Regressor:
 n_estimators is 10
 depth is 5
 MSE is 28.305789513972115
Regressor:
 n_estimators is 10
 depth is 10
 MSE is 28.367223723407577
Regressor:
 n_estimators is 10
 depth is 100
 MSE is 28.367223723407577
Regressor:
 n_estimators is 50
 depth is 1
 MSE is 40.201523385516744
Regressor:
 n_estimators is 50
 depth is 2
 MSE is 26.559115126129083
Regressor:
 n_estimators is 50
 depth is 5
 MSE is 24.267424605727427
Regressor:
 n_estimators is 50
 depth is 10
 MSE is 26.137867546568163
Regressor:
 n_estimators is 50
 depth is 100
 MSE is 25.324815651627866
Regressor:
 n_estimators is 100
 depth is 1
 MSE is 34.1719010687726
Regressor:
 n_estimators is 100
 depth is 2
 MSE is 26.96708376323875
Regressor:
 n_estimators is 100
 depth is 5
 MSE is 23.842264717606273
Regressor:
 n_estimators is 100
 depth is 10
 MSE is 27.525827877184835
Regressor:
 

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [23]:
from sklearn import linear_model

In [25]:
lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_train, y_train)
ans5 = metrics.mean_squared_error(lin_reg.predict(X_test), y_test)**0.5
print("Lin. regression error is {}".format(ans5))
write_ans(5, ans5)

Lin. regression error is 8.270468034938046
